In [8]:
import pandas as pd

In [9]:
df_lego = pd.read_csv('../00_CSV_Descargado/df_lego_work.csv')

df_lego.head()

,SetID,Number,Variant,YearFrom,Category,Theme,ThemeGroup,Subtheme,SetName,Image,...,BrickLinkSoldPriceNew,BrickLinkSoldPriceUsed,Designers,LaunchDate,ExitDate,QtyOwned,QtyOwnedNew,QtyOwnedUsed,QtyWanted,WantedPriority
0,7530,10,2,1973,Normal,Duplo,Pre-school,NaN,Pre-School Set,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
1,7531,20,2,1973,Normal,Duplo,Pre-school,NaN,Building Set,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
2,1119,28,1,1979,Normal,Duplo,Pre-school,NaN,Nursery Furniture,X,...,NaN,11.72,NaN,NaN,NaN,0,0,0,0,0
3,7532,30,2,1973,Normal,Duplo,Pre-school,NaN,Building Set,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
4,1382,32,1,1979,Normal,Duplo,Pre-school,NaN,Living Room Furniture,X,...,NaN,10.52,NaN,NaN,NaN,0,0,0,0,0


***Antes de crear columnas voy a limpiar los nulos y las columnas***

In [10]:
null_counts = df_lego.isnull().sum()

In [11]:
# Veo solo las columnas que tienen valores nulos
null_counts = null_counts[null_counts > 0]

In [12]:
null_counts

ThemeGroup                   4
Subtheme                  1448
Image                      486
ImageFilename              486
USRetailPrice             2741
UKRetailPrice             3397
CARetailPrice             4686
DERetailPrice             5142
USDateAdded               4419
USDateRemoved             4419
Pieces                     301
Minifigs                  3322
USItemNumber              2826
EUItemNumber              3047
EAN                       3887
UPC                       4262
Width                     3049
Height                    3050
Depth                     3053
Weight                    6334
AgeMin                    3587
AgeMax                    7160
BrickLinkSoldPriceNew     1357
BrickLinkSoldPriceUsed    2464
Designers                 6508
LaunchDate                3380
ExitDate                  3410
dtype: int64

In [13]:
# Reemplazo los valores nulos en 'Subtheme' por 'Unknown'
df_lego['Subtheme'] = df_lego['Subtheme'].fillna('Unknown')

In [ ]:
df_lego.head()